In [2]:
import pandas as pd
import numpy as np
import osmnx as ox
import pydeck as pdk

# Layer Unici

## BUS

In [3]:
places=["Milan,Lombardy,Italy","Segrate,Lombardy,Italy"]
tags = {"public_transport":"platform",
        "bus":"yes",
        "node":True}

all_pois = ox.geometries_from_place(places,tags)
bus=pd.DataFrame(all_pois)
bus=bus[["name","public_transport","geometry"]].reset_index()
bus=bus[bus.element_type=="node"]
bus["lat"]=bus.geometry.apply(lambda x:str(x).split("(")[1].replace(")","").split(" ")[1])
bus["lon"]=bus.geometry.apply(lambda x:str(x).split("(")[1].replace(")","").split(" ")[0])
classe=["Bus"]*len(bus)
bus["Classe"]=classe
bus=bus[["Classe","lat","lon","name"]]
bus.head()

# Treni e Metro

In [8]:
tags2 = {"public_transport":"station"}

all_pois2 = ox.geometries_from_place(places,tags2)
point2=pd.DataFrame(all_pois2).reset_index()
point2=point2[point2.element_type=="node"]
point2=point2[["name","public_transport","geometry","subway","railway"]].reset_index()

point2["lat"]=point2.geometry.apply(lambda x:str(x).split("(")[1].replace(")","").split(" ")[1])
point2["lon"]=point2.geometry.apply(lambda x:str(x).split("(")[1].replace(")","").split(" ")[0])

train=point2[point2.subway!="yes"]
metro=point2[point2.subway=="yes"]
classe=["Metro"]*len(metro)
metro["Classe"]=classe
classe=["Train"]*len(train)
train["Classe"]=classe
train=train[["Classe","lat","lon","name"]]
print(train.head())
metro=metro[["Classe","lat","lon","name"]]
print(metro.head())

  Classe         lat        lon                                 name
0  Train  45.4684048   9.175523                       Milano Cadorna
1  Train    45.48453   9.187309  Milano Porta Garibaldi (superficie)
2  Train  45.4423024   9.130128                Milano San Cristoforo
3  Train  45.5128565  9.2143385                 Milano Greco Pirelli
4  Train  45.4854753  9.2376864                      Milano Lambrate
   Classe         lat        lon                name
23  Metro  45.4641385  9.1885548               Duomo
26  Metro  45.4553979  9.1129252           Bisceglie
27  Metro  45.4368888  9.1681331           Famagosta
29  Metro  45.4335776  9.2380577         Rogoredo FS
49  Metro  45.4643154  9.2363556  Stazione Forlanini


<ipython-input-8-b1771235205f>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metro["Classe"]=classe
<ipython-input-8-b1771235205f>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["Classe"]=classe


## Parcheggi

In [ ]:
tags3 = {"amenity":"parking"}

all_pois3 = ox.geometries_from_place(places,tags3)
parcheggi=pd.DataFrame(all_pois3).reset_index()
parcheggi_nodi=parcheggi[parcheggi.element_type=="node"]
parcheggi_way=parcheggi[parcheggi.element_type=="way"]
parcheggi_relation=parcheggi[parcheggi.element_type=="relation"]
parcheggi_nodi=parcheggi_nodi[["name","geometry"]]
parcheggi_nodi["lat"]=parcheggi_nodi.geometry.apply(lambda x:str(x).split("(")[1].replace(")","").split(" ")[1])
parcheggi_nodi["lon"]=parcheggi_nodi.geometry.apply(lambda x:str(x).split("(")[1].replace(")","").split(" ")[0])

parcheggi_way=parcheggi_way[["name","geometry"]]
parcheggi_way["lat"]=parcheggi_way.geometry.apply(lambda x:str(x).split("(")[2].split(",")[0].split(" ")[0])
parcheggi_way["lon"]=parcheggi_way.geometry.apply(lambda x:str(x).split("(")[2].split(",")[0].split(" ")[1])
parcheggi_relation=parcheggi_relation[["name","geometry"]]
parcheggi_relation["split"]=parcheggi_relation.geometry.apply(lambda x:str(x).split("(")[0])
parcheggi_polygon=parcheggi_relation[parcheggi_relation.split=="POLYGON "]
parcheggi_multipolygon=parcheggi_relation[parcheggi_relation.split=="MULTIPOLYGON "]
parcheggi_polygon["lat"]=parcheggi_polygon.geometry.apply(lambda x:str(x).split("(")[2].split(",")[0].split(" ")[1])
parcheggi_polygon["lon"]=parcheggi_polygon.geometry.apply(lambda x:str(x).split("(")[2].split(",")[0].split(" ")[0])
parcheggi_polygon=parcheggi_polygon[["name","geometry","lat","lon"]]
parcheggi_multipolygon["lat"]=parcheggi_multipolygon.geometry.apply(lambda x:str(x).split("(")[3].split(",")[0].split(" ")[1])
parcheggi_multipolygon["lon"]=parcheggi_multipolygon.geometry.apply(lambda x:str(x).split("(")[3].split(",")[0].split(" ")[0])
parcheggi_multipolygon=parcheggi_multipolygon[["name","geometry","lat","lon"]]
parcheggi_complete=pd.concat([parcheggi_nodi, parcheggi_way,parcheggi_polygon,parcheggi_multipolygon], ignore_index=True)
classe=["Parcheggio"]*len(parcheggi_complete)
parcheggi_complete["Classe"]=classe
parcheggi_complete=parcheggi_complete[["Classe","lat","lon","name"]]
parcheggi_complete.head()

## Dataset Unico

In [103]:
map_data=pd.concat([parcheggi_complete, bus,train,metro], ignore_index=True)

## Icone

In [19]:
Bus_icon = "https://upload.wikimedia.org/wikipedia/commons/b/b3/Map_marker_icon_%E2%80%93_Nicolas_Mollet_%E2%80%93_Bus_Stop_%E2%80%93_Transportation_%E2%80%93_Light.png"

Bus_icon_data = {

    "url": Bus_icon,
    "width": 32,
    "height": 37,
    "anchorY": 'auto',
}


In [20]:
Metro_icon = "https://upload.wikimedia.org/wikipedia/commons/5/57/Map_marker_icon_%E2%80%93_Nicolas_Mollet_%E2%80%93_Subway_%E2%80%93_Transportation_%E2%80%93_Light.png"

Metro_icon_data = {

    "url": Metro_icon,
    "width": 32,
    "height": 37,
    "anchorY": 'auto',
}

In [21]:
Train_icon = "https://upload.wikimedia.org/wikipedia/commons/1/16/Map_marker_icon_%E2%80%93_Nicolas_Mollet_%E2%80%93_Train_%E2%80%93_Transportation_%E2%80%93_Light.png"

Train_icon_data = {

    "url": Train_icon,
    "width": 32,
    "height": 37,
    "anchorY": 'auto',
}

In [22]:
Parcheggi_icon = "https://upload.wikimedia.org/wikipedia/commons/c/c7/Map_marker_icon_%E2%80%93_Nicolas_Mollet_%E2%80%93_Parking_%E2%80%93_Transportation_%E2%80%93_Light.png"

Parcheggi_icon_data = {

    "url": Parcheggi_icon,
    "width": 32,
    "height": 37,
    "anchorY": 'auto',
}

# Assegno le icone alle Classi

In [109]:
icon_data = []
for index,row in map_data.iterrows():
  if row["Classe"]=="Parcheggio":
    icon_data.append(Parcheggi_icon_data)
  elif row["Classe"]=="Bus":
    icon_data.append(Bus_icon_data)
  elif row["Classe"]=="Train":
    icon_data.append(Train_icon_data)
  elif row["Classe"]=="Metro":
    icon_data.append(Metro_icon_data)   

In [110]:
map_data["icon_data"]=icon_data
map_data["lat"]=map_data["lat"].apply(lambda x : float(x))
map_data["lon"]=map_data["lon"].apply(lambda x : float(x))

## Preparazione del layer

Tutti i mezzi

In [63]:
Nav_Points = pdk.Layer(
    type="IconLayer",
    data=map_data,
    get_icon="icon_data",
    get_size=10,
    size_scale=3,
    get_position=["lon", "lat"],
    pickable=True,
)

Origin Layer e Initial State


In [64]:
import pandas as pd
import geopandas
import folium
import matplotlib.pyplot as plt
import pydeck as pdk
INITIAL_VIEW_STATE = pdk.ViewState(
  latitude=45.47185532715593, 
  longitude=9.275071955673953,
  zoom=10,
  max_zoom=16,
  pitch=45,
  bearing=0
)




origin=pd.DataFrame({"lon":9.275071955673953,"lat":45.47185532715593,"name":"Esposizioni Novegro"}, index=[0])
import pydeck as pdk
import pandas as pd


# Data from OpenStreetMap, accessed via osmpy

ICON_URL = "https://upload.wikimedia.org/wikipedia/commons/9/9e/Map_marker_icon_%E2%80%93_Nicolas_Mollet_%E2%80%93_Pin_%E2%80%93_Tourism_%E2%80%93_Light.png"

icon_data = {

    "url": ICON_URL,
    "width": 32,
    "height": 37,
    "anchorY": 'auto',
}


data=origin

data["icon_data"] = None
for i in data.index:
    data["icon_data"][i] = icon_data

data=data[["lat","lon","name","icon_data"]]



view_state = INITIAL_VIEW_STATE

origin_layer = pdk.Layer(
    type="IconLayer",
    data=data,
    get_icon="icon_data",
    get_size=10,
    size_scale=3,
    get_position=["lon", "lat"],
    pickable=True,
)

<ipython-input-64-17d9ee861fef>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["icon_data"][i] = icon_data


# Mappa

In [ ]:
pdk.Deck(layers=[Nav_Points,origin_layer], map_style='road',initial_view_state=INITIAL_VIEW_STATE, tooltip={"text": "{name}"})

## TEST PRESONALIZZAZIONE

In [117]:
a=input()
b=input()
c=input()
d=input()


Metro
Parcheggio



In [119]:
list_values=[a,b,c,d]
print(list_values)
test=map_data[map_data.Classe.isin(list_values)]
test.Classe.unique()

['', 'Metro', 'Parcheggio', '']


array(['Parcheggio', 'Metro'], dtype=object)

In [120]:
Nav_Points_part = pdk.Layer(
    type="IconLayer",
    data=test,
    get_icon="icon_data",
    get_size=10,
    size_scale=3,
    get_position=["lon", "lat"],
    pickable=True,
)

In [121]:
pdk.Deck(layers=[Nav_Points_part,origin_layer], map_style='road',initial_view_state=INITIAL_VIEW_STATE, tooltip={"text": "{name}"})

Output hidden; open in https://colab.research.google.com to view.